# Diffusion Music Generation Level 1

### imports

In [ ]:
import IPython

import torch
from torch.utils.data import DataLoader
import yaml
import matplotlib.pyplot as plt
import torchaudio

from models.multi_level_vqvae import MultiLvlVQVariationalAutoEncoder
from models.diffusion_vit import DiffusionViTSongCond
from models.diffusion_unet import WaveNetDiffusion
from models.transformer_vqvae import TransformerVQVAE
from loaders.latent_loaders import Lvl2InputDataset
from utils.other import load_cfg_dict

### Load configuration files and add weight paths

In [ ]:
config_path = "config/lvl1_config.yaml"
weights_path = "model_weights/lvl1_vqvae.ckpt"
cfg = load_cfg_dict(config_path)

config_path_diff = "config/diff_lvl1_config.yaml"
weights_path_diff = "model_weights/lvl1_diff.ckpt"
cfg_diff = load_cfg_dict(config_path_diff)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Load the models

In [ ]:
model_vqvae = MultiLvlVQVariationalAutoEncoder.load_from_checkpoint(weights_path, **cfg, strict=False).to(device=device)
model_vqvae.eval()

model_diffusion = DiffusionViTSongCond.load_from_checkpoint(weights_path_diff, **cfg_diff, strict=False).to(device=device)
model_diffusion.eval()

### Define the player

In [ ]:
# this is a wrapper that take a filename and publish an html <audio> tag to listen to it

def wavPlayer(filepath):
    """ will display html 5 player for compatible browser

    Parameters :
    ------------
    filepath : relative filepath with respect to the notebook directory ( where the .ipynb are not cwd)
               of the file to play

    The browser need to know how to play wav through html5.

    there is no autoplay to prevent file playing when the browser opens
    """
    
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
      <source src="files/%s" type="audio/mp3" />
      Your browser does not support the audio element.
    </audio>
    </body>
    """%(filepath)
    display(HTML(src))

## Load the diffusion dataset and display the latents

In [ ]:
dataset = Lvl2InputDataset(preload=True, **cfg_diff)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for sample in dataloader:
    lvl2_sample = sample['music slice'].squeeze(0)
    print(f"Current track is {sample['track name'][0]}")
    break

plt.figure(figsize=(25, 5))
plt.matshow(lvl2_sample.cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-5, vmax=5.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Run diffusion model

In [ ]:
with torch.no_grad():
    random_noise_start = torch.randn_like(lvl2_sample)
    sample_with_random = random_noise_start * 0.0 + lvl2_sample * 1.0
    lvl2_latent_gen = model_diffusion.denoise(sample_with_random.unsqueeze(0).to(device), {1: 2, 0: 50}).squeeze(0)

plt.figure(figsize=(25, 5))
plt.matshow(lvl2_latent_gen.cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-5, vmax=5.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Generate music

In [ ]:
music_generated, _ = model_vqvae.decode(lvl2_latent_gen.unsqueeze(0).to(device))
music_sample_rec = music_generated.view((1, -1))
plt.figure(figsize=(25, 5))
plt.plot(music_sample_rec[0, ...].cpu().detach().numpy())
plt.ylim((-1.1, 1.1))
plt.show()
torchaudio.save('sample_out.mp3', music_sample_rec.cpu().detach(), 44100, format='mp3')
IPython.display.Audio(filename="sample_out.mp3")